In [1]:
from SynTemp.SynUtils.utils import load_database, load_from_pickle
data = load_from_pickle('./Data/uspto/uspto_its_graph_rules_cluster.pkl.gz')

In [2]:
from SynTemp.SynMØD.naive_cluster import NaiveCluster
node_label_names = ["element", "charge"]
naive_cluster = NaiveCluster(node_label_names=node_label_names, node_label_default=["*", 0], edge_attribute="order")
its_graph_rules_cluster = naive_cluster.process_rules_clustering(data, rule_column='GraphRules')

In [3]:
its_graph_rules_cluster[0:2]

[{'R-id': 'USPTO_50K_26',
  'ITSGraph': (<networkx.classes.graph.Graph at 0x7f721c9046d0>,
   <networkx.classes.graph.Graph at 0x7f71fb08ab50>),
  'GraphRules': (<networkx.classes.graph.Graph at 0x7f71fb09a650>,
   <networkx.classes.graph.Graph at 0x7f71fb09ae90>),
  'naive_cluster': 0,
  'Reaction Type': 'Single Cyclic',
  'Rings': [4]},
 {'R-id': 'USPTO_50K_50',
  'ITSGraph': (<networkx.classes.graph.Graph at 0x7f71fb09b750>,
   <networkx.classes.graph.Graph at 0x7f71fb0ab8d0>),
  'GraphRules': (<networkx.classes.graph.Graph at 0x7f71fb0ba790>,
   <networkx.classes.graph.Graph at 0x7f71fb0bafd0>),
  'naive_cluster': 1,
  'Reaction Type': 'Single Cyclic',
  'Rings': [4]}]

In [4]:
import pandas as pd
pd.DataFrame(its_graph_rules_cluster)['Rings'].value_counts()

Rings
[4]                      26348
[]                        1061
[5]                        692
[6]                        297
[3, 5]                     169
[4, 4, 6]                  151
[3]                         67
[4, 4]                      50
[7]                         22
[4, 5, 5, 5, 6, 7, 8]       20
[3, 3]                      14
[5, 6, 7]                    5
[8]                          4
[4, 4, 4]                    2
[3, 3, 3, 4, 4, 5]           2
[3, 4, 5]                    2
[4, 5, 5]                    1
Name: count, dtype: int64